### Logic Programming

In [1]:
from logpy import run, var, fact
import logpy.assoccomm as la

In [2]:
#Define mathematical operations
add = 'addition'
mul = 'multiplication'

In [3]:
#Declare that these operations are commutative and associative using the fact's system
fact(la.commutative, add)
fact(la.commutative, mul)

fact(la.associative, add)
fact(la.associative, mul)

In [4]:
#Define some variables
a, b, c = var('a'), var('b'), var('c')

In [5]:
#Original expression
#expression_orig = 3 x (-2) + (1 + 2 x 3) x -1
#expression1 = (1 + 2xa)xb + 3xc
#expression2 = cx3 + bx(2xa + 1)
#expression3 = (((2xa)xb) + xb) + 3xc

expression_orig = (add, (mul, 3, -2), (mul, (add, 1, (mul, 2, 3)), -1))
expression1 = (add, (mul, (add, 1, (mul, 2, a)), b), (mul, 3, c))
expression2 = (add, (mul, c, 3), (mul, b, (add, (mul, 2, a), 1)))
expression3 = (add, (add, (mul, (mul, 2, a), b), b), (mul, 3, c))

In [6]:
#Compare expressions
print(run(0, (a, b, c), la.eq_assoccomm(expression1, expression_orig)))
print(run(0, (a, b, c), la.eq_assoccomm(expression2, expression_orig)))
print(run(0, (a, b, c), la.eq_assoccomm(expression3, expression_orig)))

((3, -1, -2),)
((3, -1, -2),)
()


### Validating primes

In [7]:
import itertools as it
import logpy.core as lc
from logpy import membero
from sympy.ntheory.generate import prime, isprime

In [8]:
#Check if the elements of x are prime
def check_prime(x):
    if lc.isvar(x):
        return lc.condeseq([(lc.eq, x, p)] for p in map(prime, it.count(1)))
    else:
        return lc.success if isprime(x) else lc.fail

In [9]:
#Declare the variable
x = lc.var()

In [45]:
#Check if an element in the list is a prime number
list_nums = (23, 4, 27, 17, 13, 10, 21, 29, 3, 32, 11, 19)

print('List of primes in the list:')
print(set(lc.run(0, x, (membero, x, list_nums), (check_prime, x))))

List of primes in the list:
{3, 11, 13, 17, 19, 23, 29}


In [11]:
#Print the first 7 prime numbers
print('List of first 7 prime numbers')
print(lc.run(7, x, check_prime(x)))

List of first 7 prime numbers
(2, 3, 5, 7, 11, 13, 17)


### Parsing a family tree

In [12]:
import json
from logpy import Relation, facts, run, conde, var, eq

In [13]:
#Check if x is the parent of y
def parent(x, y):
    return conde([father(x, y)], [mother(x, y)])

In [14]:
#Check if x is the grandparent of y
def grandparent(x, y):
    temp = var()
    return conde((parent(x, temp), parent(temp, y)))

In [15]:
#Check for sibling relationship between x and y
def sibling(x, y):
    temp = var()
    return conde((parent(temp, x), parent(temp, y)))

In [16]:
#Check if x is y's uncle
def uncle(x, y):
    temp = var()
    return conde((father(temp, x), grandparent(temp, y)))

In [17]:
father = Relation()
mother = Relation()

In [18]:
with open('data/relationships.json') as f:
    d = json.loads(f.read())
    
for item in d['father']:
    facts(father, (list(item.keys())[0], list(item.values())[0]))
for item in d['mother']:
    facts(mother, (list(item.keys())[0], list(item.values())[0]))

In [19]:
#Declare variable x
x = var()

In [20]:
#Find John's children
name = 'John'
output = run(0, x, father(name, x))

print(f"List of {name}'s children")
for item in output:
    print(item)

List of John's children
William
David
Adam


In [21]:
#Find William's mother
name = 'William'
output = run(0, x, mother(x, name))[0]

print(f"Name of {name}'s mother: {output}")

Name of William's mother: Megan


In [22]:
#Find Adam's parents
name = 'Adam'
output = run(0, x, parent(x, name))

print(f"List of {name}'s parents")
for item in output:
    print(item)

List of Adam's parents
John
Megan


In [23]:
#Find Wayne's grandparents
name = 'Wayne'
output = run(0, x, grandparent(x, name))

print(f"List of {name}'s grandparents")
for item in output:
    print(item)

List of Wayne's grandparents
John
Megan


In [24]:
#Find Megan's grandchildren
name = 'Megan'
output = run(0, x, grandparent(name, x))

print(f"List of {name}'s grandchildren")
for item in output:
    print(item)

List of Megan's grandchildren
Sophia
Stephanie
Peter
Chris
Wayne
Julie
Tiffany
Neil


In [26]:
#Find David's siblings
name = 'David'

output = run(0, x, sibling(x, name))
siblings = [x for x in output if x != name]

print(f"List of {name}'s siblings")
for item in siblings:
    print(item)

List of David's siblings
William
Adam


In [27]:
#Find Tiffany's uncles
name = 'Tiffany'
name_father = run(0, x, father(x, name))[0]

output = run(0, x, uncle(x, name))
output = [x for x in output if x != name_father]

print(f"List of {name}'s uncles")
for item in output:
    print(item)

List of Tiffany's uncles
William
Adam


In [28]:
#Find all spouses
a, b, c = var(), var(), var()
output = run(0, (a, b), father(a, c), mother(b, c))

print('List of spouses')
for husband, wife in output:
    print(f'Husband: {husband} <==> Wife: {wife}')

List of spouses
Husband: David <==> Wife: Olivia
Husband: John <==> Wife: Megan
Husband: William <==> Wife: Emma
Husband: Adam <==> Wife: Lily


### Analyzing Geography

In [29]:
adjacent = Relation()
coastal = Relation()

In [30]:
file_coastal = 'data/coastal_states.txt'

with open(file_coastal) as f:
    line = f.read()
    coastal_states = line.split(',')
    
#Add the info to the fact base
for state in coastal_states:
    fact(coastal, state)

In [31]:
file_adjacent = 'data/adjacent_states.txt'

with open(file_adjacent) as f:
    adjlist = [line.strip().split(',') for line in f if line and line[0].isalpha()]
    
#Add the info to the fact base
for L in adjlist:
    head, tail = L[0], L[1:]
    for state in tail:
        fact(adjacent, head, state)

In [32]:
#Initialize the variables
x = var()
y = var()

In [33]:
#Check if Nevada is adjacent to Louisiana
output = run(0, x, adjacent('Nevada', 'Louisiana'))

print('Is Nevada adjacent to Louisiana?')
print('Yes' if len(output) else 'No')

Is Nevada adjacent to Louisiana?
No


In [34]:
#Print states adjacent to Oregon
output = run(0, x, adjacent('Oregon', x))

print('List of states adjacent to Oregon:')
for item in output:
    print(item)

List of states adjacent to Oregon:
Washington
Nevada
California
Idaho


In [39]:
#Print states adjacent to Mississippi that are coastal
output = run(0, x, adjacent('Mississippi', x), coastal(x))

print('List of coastal states adjacent to Mississippi')
for item in output:
    print(item)

List of coastal states adjacent to Mississippi
Alabama
Louisiana


In [37]:
#List of 'n' states that border a coastal state - n = 7 in this case
n = 7
output = run(n, x, coastal(y), adjacent(x, y))

print(f'List of {n} states that border a coastal state:')
for item in output:
    print(item)

List of 7 states that border a coastal state:
Oregon
Connecticut
Mississippi
North Carolina
Virginia
New Jersey
Pennsylvania


In [40]:
#List of states that are adjacent to both Arkansas and Kentucky
output = run(0, x, adjacent('Arkansas', x), adjacent('Kentucky', x))

print('List of states that are adjacent to both Arkansas and Kentucky')
for item in output:
    print(item)

List of states that are adjacent to both Arkansas and Kentucky
Missouri
Tennessee


### Building a puzzle solver

In [41]:
from logpy.core import lall

In [43]:
people = var()

In [46]:
#There are four people
rules = lall(
    (eq, (var(), var(), var(), var()), people), # There are 4 people
    (membero, ('Steve', var(), 'blue', var()), people), # Steve's car is blue
    (membero, (var(), 'cat', var(), 'Canada'), people), # Person who has a cat lives in Canada
    (membero, ('Matthew', var(), var(), 'USA'), people), # Matthew lives in USA
    (membero, (var(), var(), 'black', 'Australia'), people), # The person who has a black car lives in Australia 
    (membero, ('Jack', 'cat', var(), var()), people), # Jack has a cat
    (membero, ('Alfred', var(), var(), 'Australia'), people), # Alfred lives in Australia
    (membero, (var(), 'dog', var(), 'France'), people), # Person who owns the dog lives in France
    (membero, (var(), 'rabbit', var(), var()), people) # Who has a rabbit?
)

In [51]:
solutions = run(0, people, rules)

#Extract the output 
output = [answer for answer in solutions[0] if 'rabbit' in answer][0][0]

In [56]:
#print the output
print(f'{output} is the owner of the rabbit')

print('\nHere are all the details:')
attribs = ['Name', 'Pet', 'Color', 'Country']
print('\n' + '\t\t'.join(attribs))

print('==' * 30)
for item in solutions[0]:
    print('\t\t'.join([str(x) for x in item]))

Matthew is the owner of the rabbit

Here are all the details:

Name		Pet		Color		Country
Steve		dog		blue		France
Jack		cat		~_112		Canada
Matthew		rabbit		~_114		USA
Alfred		~_116		black		Australia
